In [1]:
from pathlib import Path
import pandas as pd
import re

TSV = Path("../data/localsample/meta_500_20_4_10_all.tsv")
FASTA_IN = Path("../data/fulldesign_2019-02-27_wGBKsw.fasta")
FASTA_OUT = Path("../data/localsample/500_design.fasta")

In [23]:
df = pd.read_csv(TSV, sep="\t")
headers_needed = set(df["FullName"].dropna().unique().tolist())
print(len(headers_needed))

header_pattern = re.compile(r"^>ID=([^\s]+)\s+AC=([^\s]+)\s+OXX=([^\s]+)\s*$")

def extract_header(header: str):
    if pd.isna(header):
        return None
    match_ = header_pattern.match(str(header))
    return header.split(">")[1] if match_ else None

def write_fasta(fasta_in, fasta_out, want_headers):
    with fasta_in.open("r", encoding="utf-8", errors="replace") as fin, \
         fasta_out.open("w", encoding="utf-8") as fout:
        
        headers_added = set()
        keep = False
        current_header = None
        for line in fin:
            if line.startswith(">"):
                current_header = extract_header(line.rstrip("\n"))
                keep = current_header in want_headers
                if keep and current_header not in headers_added:
                    headers_added.add(current_header)
                    fout.write(line)
                else:
                    keep = False
            else:
                if keep:
                    fout.write(line)

273


In [24]:
write_fasta(FASTA_IN, FASTA_OUT, headers_needed)